# 3DCT高速化に向けた NumPyブロードキャストに関する検証1
- 問題 : d/W(a,b,c) = d - x(a)cos(c) + y(b)sin(c) を行列演算で行いたい

In [16]:
import numpy as np

a = np.arange(3).reshape((1,3)).astype(str)
a = a.astype(object)
a = a + 'a '
a

array([['0a ', '1a ', '2a ']], dtype=object)

- 注意 : numpyでは文字列の結合を"+"でできないので、str型で配列を作ってから、結合前にnp.astype(object)でpythonのオブジェクトに中身を変換

In [17]:
b = np.arange(3).reshape((1,3)).astype(str)
b = b.astype(object)
b = b + 'b '
b

array([['0b ', '1b ', '2b ']], dtype=object)

In [22]:
c = np.arange(3).reshape((3,1)).astype(str)
c = c.astype(object)
c = c + 'c'
c.reshape((1,3))
c

array([['0c'],
       ['1c'],
       ['2c']], dtype=object)

In [23]:
ac = a + c
ac

array([['0a 0c', '1a 0c', '2a 0c'],
       ['0a 1c', '1a 1c', '2a 1c'],
       ['0a 2c', '1a 2c', '2a 2c']], dtype=object)

In [25]:
bc = b + c
bc

array([['0b 0c', '1b 0c', '2b 0c'],
       ['0b 1c', '1b 1c', '2b 1c'],
       ['0b 2c', '1b 2c', '2b 2c']], dtype=object)

- ac --> x(a)cos(c)
- bc --> y(b)sin(c)
    - つまりこいつらの足し算で Cの番号 が共通かつ、各Cで (a,b)の2次元行列になってればおｋ

In [29]:
ac = ac.reshape((3,3,1))
ac

array([[['0a 0c'],
        ['1a 0c'],
        ['2a 0c']],

       [['0a 1c'],
        ['1a 1c'],
        ['2a 1c']],

       [['0a 2c'],
        ['1a 2c'],
        ['2a 2c']]], dtype=object)

In [31]:
bc = bc.reshape((3,1,3))
bc

array([[['0b 0c', '1b 0c', '2b 0c']],

       [['0b 1c', '1b 1c', '2b 1c']],

       [['0b 2c', '1b 2c', '2b 2c']]], dtype=object)

In [33]:
abc = ac + bc
abc

array([[['0a 0c0b 0c', '0a 0c1b 0c', '0a 0c2b 0c'],
        ['1a 0c0b 0c', '1a 0c1b 0c', '1a 0c2b 0c'],
        ['2a 0c0b 0c', '2a 0c1b 0c', '2a 0c2b 0c']],

       [['0a 1c0b 1c', '0a 1c1b 1c', '0a 1c2b 1c'],
        ['1a 1c0b 1c', '1a 1c1b 1c', '1a 1c2b 1c'],
        ['2a 1c0b 1c', '2a 1c1b 1c', '2a 1c2b 1c']],

       [['0a 2c0b 2c', '0a 2c1b 2c', '0a 2c2b 2c'],
        ['1a 2c0b 2c', '1a 2c1b 2c', '1a 2c2b 2c'],
        ['2a 2c0b 2c', '2a 2c1b 2c', '2a 2c2b 2c']]], dtype=object)

In [34]:
abc[0,1,2]

'1a 0c2b 0c'

In [35]:
abc[2,0,1]

'0a 2c1b 2c'

- 要素の形成はできたが問題あり
    - d/W(c,a,b) になってしまっている
- np.transposeで入れ替えることができる

In [40]:
result = abc.transpose(1,2,0)
result

array([[['0a 0c0b 0c', '0a 1c0b 1c', '0a 2c0b 2c'],
        ['0a 0c1b 0c', '0a 1c1b 1c', '0a 2c1b 2c'],
        ['0a 0c2b 0c', '0a 1c2b 1c', '0a 2c2b 2c']],

       [['1a 0c0b 0c', '1a 1c0b 1c', '1a 2c0b 2c'],
        ['1a 0c1b 0c', '1a 1c1b 1c', '1a 2c1b 2c'],
        ['1a 0c2b 0c', '1a 1c2b 1c', '1a 2c2b 2c']],

       [['2a 0c0b 0c', '2a 1c0b 1c', '2a 2c0b 2c'],
        ['2a 0c1b 0c', '2a 1c1b 1c', '2a 2c1b 2c'],
        ['2a 0c2b 0c', '2a 1c2b 1c', '2a 2c2b 2c']]], dtype=object)

In [41]:
result[0,1,2]

'0a 2c1b 2c'

## できれば計算過程追いやすくしたい
- 今のacの配置 --> (c,a)

In [43]:
ac[1,2,0]

'2a 1c'

- ブロードキャスト演算を逆にしたら？

In [45]:
ca = c + a
ca = ca.reshape((3,3,1))
ca[1,2,0]

'1c2a '

In [46]:
a_dash = a.reshape((3,1))
c_dash = c.reshape((1,3))
ca = a_dash + c_dash
ca = ca.reshape((3,3,1))
ca[1,2,0]

'1a 2c'

In [48]:
ca = ca.reshape((3,1,3))
ca[1,0,2]

'1a 2c'

### つまり最初の１次元配列のreshapeが間違っていた

In [49]:
ac_dash = ca

b_dash = b.reshape((3,1))
bc_dash = b_dash + c_dash
bc_dash = bc_dash.reshape((1,3,3))
bc_dash

array([[['0b 0c', '0b 1c', '0b 2c'],
        ['1b 0c', '1b 1c', '1b 2c'],
        ['2b 0c', '2b 1c', '2b 2c']]], dtype=object)

In [50]:
bc_dash[0,1,2]

'1b 2c'

In [51]:
abc_dash = ac_dash + bc_dash
abc_dash

array([[['0a 0c0b 0c', '0a 1c0b 1c', '0a 2c0b 2c'],
        ['0a 0c1b 0c', '0a 1c1b 1c', '0a 2c1b 2c'],
        ['0a 0c2b 0c', '0a 1c2b 1c', '0a 2c2b 2c']],

       [['1a 0c0b 0c', '1a 1c0b 1c', '1a 2c0b 2c'],
        ['1a 0c1b 0c', '1a 1c1b 1c', '1a 2c1b 2c'],
        ['1a 0c2b 0c', '1a 1c2b 1c', '1a 2c2b 2c']],

       [['2a 0c0b 0c', '2a 1c0b 1c', '2a 2c0b 2c'],
        ['2a 0c1b 0c', '2a 1c1b 1c', '2a 2c1b 2c'],
        ['2a 0c2b 0c', '2a 1c2b 1c', '2a 2c2b 2c']]], dtype=object)

In [52]:
abc_dash[0,1,2]

'0a 2c1b 2c'

- 追いやすい計算過程ができた

## 実行時間を比較する

In [74]:
%%timeit
abc_forloop = np.empty((3,3,3)).astype(object)
for i in range(a_dash.shape[0]):
    for j in range(b_dash.shape[0]):
        for k in range(c_dash.shape[1]):
            abc_forloop[i,j,k] = (a_dash[i,0]+c_dash[0,k]) + (b_dash[j,0]+c_dash[0,k])

29.6 µs ± 591 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [75]:
abc_forloop

array([[['0a 0c0b 0c', '0a 1c0b 1c', '0a 2c0b 2c'],
        ['0a 0c1b 0c', '0a 1c1b 1c', '0a 2c1b 2c'],
        ['0a 0c2b 0c', '0a 1c2b 1c', '0a 2c2b 2c']],

       [['1a 0c0b 0c', '1a 1c0b 1c', '1a 2c0b 2c'],
        ['1a 0c1b 0c', '1a 1c1b 1c', '1a 2c1b 2c'],
        ['1a 0c2b 0c', '1a 1c2b 1c', '1a 2c2b 2c']],

       [['2a 0c0b 0c', '2a 1c0b 1c', '2a 2c0b 2c'],
        ['2a 0c1b 0c', '2a 1c1b 1c', '2a 2c1b 2c'],
        ['2a 0c2b 0c', '2a 1c2b 1c', '2a 2c2b 2c']]], dtype=object)

In [76]:
%%timeit
ac_dash = a_dash + c_dash
ac_dash = ac_dash.reshape((3,1,3))
bc_dash = b_dash + c_dash
bc_dash = bc_dash.reshape((1,3,3))
abc_dash = ac_dash + bc_dash

12 µs ± 41.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [77]:
abc_dash

array([[['0a 0c0b 0c', '0a 1c0b 1c', '0a 2c0b 2c'],
        ['0a 0c1b 0c', '0a 1c1b 1c', '0a 2c1b 2c'],
        ['0a 0c2b 0c', '0a 1c2b 1c', '0a 2c2b 2c']],

       [['1a 0c0b 0c', '1a 1c0b 1c', '1a 2c0b 2c'],
        ['1a 0c1b 0c', '1a 1c1b 1c', '1a 2c1b 2c'],
        ['1a 0c2b 0c', '1a 1c2b 1c', '1a 2c2b 2c']],

       [['2a 0c0b 0c', '2a 1c0b 1c', '2a 2c0b 2c'],
        ['2a 0c1b 0c', '2a 1c1b 1c', '2a 2c1b 2c'],
        ['2a 0c2b 0c', '2a 1c2b 1c', '2a 2c2b 2c']]], dtype=object)

In [80]:
abc_forloop == abc_dash

array([[[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]])

## まとめ
- 行列演算で問題を解くことができた。
### 手法
- 計算する配列は計算する前に`np.reshape`で成形
    - アサインしたい軸に元の要素数を、他の軸は1とする
- アサイン後、NumPyのブロードキャスト機能を利用して計算
### 実行時間
- 3x3 行列 : for-loop --> 29.6us / 本手法 --> 12us

# 実行手順
1. それぞれの行列を作成

In [85]:
x = np.arange(3).astype(str)
x = x.astype(object)
x = "X[" + x + "]"

y = np.arange(3).astype(str)
y = y.astype(object)
y = "Y[" + y + "]"

sin = np.arange(3).astype(str)
sin = sin.astype(object)
sin = "sin[" + sin + "]"

cos = np.arange(3).astype(str)
cos = cos.astype(object)
cos = "cos[" + cos + "] + "

print(x)
print(y)
print(sin)
print(cos)

['X[0]' 'X[1]' 'X[2]']
['Y[0]' 'Y[1]' 'Y[2]']
['sin[0]' 'sin[1]' 'sin[2]']
['cos[0] + ' 'cos[1] + ' 'cos[2] + ']


2. 軸をずらしてreshape

In [87]:
x = x.reshape((x.shape[0],1))
y = y.reshape((y.shape[0],1))
sin = sin.reshape((1,sin.shape[0]))
cos = cos.reshape((1,cos.shape[0]))

print(x)
print(y)
print(sin)
print(cos)

[['X[0]']
 ['X[1]']
 ['X[2]']]
[['Y[0]']
 ['Y[1]']
 ['Y[2]']]
[['sin[0]' 'sin[1]' 'sin[2]']]
[['cos[0] + ' 'cos[1] + ' 'cos[2] + ']]


3. ブロードキャスト演算

In [88]:
xcos = x + cos
ysin = y + sin

print(xcos)
print(ysin)

[['X[0]cos[0] + ' 'X[0]cos[1] + ' 'X[0]cos[2] + ']
 ['X[1]cos[0] + ' 'X[1]cos[1] + ' 'X[1]cos[2] + ']
 ['X[2]cos[0] + ' 'X[2]cos[1] + ' 'X[2]cos[2] + ']]
[['Y[0]sin[0]' 'Y[0]sin[1]' 'Y[0]sin[2]']
 ['Y[1]sin[0]' 'Y[1]sin[1]' 'Y[1]sin[2]']
 ['Y[2]sin[0]' 'Y[2]sin[1]' 'Y[2]sin[2]']]


4. 2.と3.をさらに繰り返す
(この時、indexを共有したいものは同じ軸に配置)

In [89]:
xcos = xcos.reshape((xcos.shape[0],1,xcos.shape[1]))
ysin = ysin.reshape((1,ysin.shape[0],ysin.shape[1]))

xcos = " - " + xcos

print(xcos)
print(ysin)

[[[' - X[0]cos[0] + ' ' - X[0]cos[1] + ' ' - X[0]cos[2] + ']]

 [[' - X[1]cos[0] + ' ' - X[1]cos[1] + ' ' - X[1]cos[2] + ']]

 [[' - X[2]cos[0] + ' ' - X[2]cos[1] + ' ' - X[2]cos[2] + ']]]
[[['Y[0]sin[0]' 'Y[0]sin[1]' 'Y[0]sin[2]']
  ['Y[1]sin[0]' 'Y[1]sin[1]' 'Y[1]sin[2]']
  ['Y[2]sin[0]' 'Y[2]sin[1]' 'Y[2]sin[2]']]]


In [91]:
result = xcos + ysin
result = "d" + result

print(result)
result[0,1,2]

[[['d - X[0]cos[0] + Y[0]sin[0]' 'd - X[0]cos[1] + Y[0]sin[1]'
   'd - X[0]cos[2] + Y[0]sin[2]']
  ['d - X[0]cos[0] + Y[1]sin[0]' 'd - X[0]cos[1] + Y[1]sin[1]'
   'd - X[0]cos[2] + Y[1]sin[2]']
  ['d - X[0]cos[0] + Y[2]sin[0]' 'd - X[0]cos[1] + Y[2]sin[1]'
   'd - X[0]cos[2] + Y[2]sin[2]']]

 [['d - X[1]cos[0] + Y[0]sin[0]' 'd - X[1]cos[1] + Y[0]sin[1]'
   'd - X[1]cos[2] + Y[0]sin[2]']
  ['d - X[1]cos[0] + Y[1]sin[0]' 'd - X[1]cos[1] + Y[1]sin[1]'
   'd - X[1]cos[2] + Y[1]sin[2]']
  ['d - X[1]cos[0] + Y[2]sin[0]' 'd - X[1]cos[1] + Y[2]sin[1]'
   'd - X[1]cos[2] + Y[2]sin[2]']]

 [['d - X[2]cos[0] + Y[0]sin[0]' 'd - X[2]cos[1] + Y[0]sin[1]'
   'd - X[2]cos[2] + Y[0]sin[2]']
  ['d - X[2]cos[0] + Y[1]sin[0]' 'd - X[2]cos[1] + Y[1]sin[1]'
   'd - X[2]cos[2] + Y[1]sin[2]']
  ['d - X[2]cos[0] + Y[2]sin[0]' 'd - X[2]cos[1] + Y[2]sin[1]'
   'd - X[2]cos[2] + Y[2]sin[2]']]]


'd - X[0]cos[2] + Y[1]sin[2]'

- point : すべての項でcosとsinのインデックスが共通している。